# Setup
**This analysis includes all projects for the 2019 Season (includes projects marked as "Dead")**

## Dependencies

In [52]:
import pandas as pd
from datetime import datetime

## Import Data

In [63]:
# imports the '[TVA] Q2 Workflow Analysis' data
all_project_data = "./data/all_projects.csv"

# imports the '[TVA] Orders Table Audit' data
all_production_data = "./data/all_production_projects.csv"

# imports '[W] ' data
improvements_data = "./data/improvement_table.csv"

# Parsing Data

## Project Data

### All Q2 Projects

**Includes projects marked as 'Dead'**

In [91]:
project_df = pd.read_csv(
    all_project_data, dtype={'Claim #': str, 'Job #': str, 'Branch' : str, 'Claim Status': str},
    parse_dates=[
        'Claim # Date',
        'FTA Scope. Req Date',
        'Submit for Estimate Date',
        '[OB] Created Scope Calc',
        '[B] Created Estimate Date',
        'Job Submittal Date',
        '[B] - Date Approved by BC',
        '[OB] Completed']
)

# provides a summary of the 'project_df' holding all jobs for the season
project_summary = project_df.count()
project_summary

Claim #                      1512
Job #                         897
Branch                       1520
Claim # Date                 1520
FTA Scope. Req Date          1520
Submit for Estimate Date     1520
[OB] Created Scope Calc      1515
[B] Created Estimate Date    1520
Job Submittal Date            972
[B] - Date Approved by BC     897
[OB] Completed                858
Claim Status                   72
dtype: int64

### All Projects in Production

**Includes projects that have completed notifying the HO of the 'Roof Start' Date**

In [107]:
production_df = pd.read_csv(
    all_production_data, dtype={'Claim #': str, 'Job #': str, 'Branch': str},
    parse_dates=[
        '[OB] Completed',
        'Permit Applied [A]',
        'Order Date',
        'Permit Received',
        'OA Date',
        'Invoice Date',
        'Ntfd H.O. Dlvry',
        'Dlvry Start',
        'Ntfd H.O. Start',
        'Roof Start']
)

# provides a summary of the 'project_df' holding all jobs for the season
production_summary = production_df.count()
production_summary

Claim #               859
Job #                 860
Branch                860
[OB] Completed        860
Permit Applied [A]    434
Order Date            685
Permit Received       398
OA Date               644
Invoice Date          579
Ntfd H.O. Dlvry       663
Dlvry Start           673
Ntfd H.O. Start       566
Roof Start            671
dtype: int64

## Rejection Data

### All Rejections

**Includes projects marked as 'Dead'**

In [108]:
# created 'improvements_df' to hold all dates of fta scope rejections to use correct date
improvements_df = pd.read_csv(
    improvements_data,
    dtype={'Claim #': str},
    parse_dates=['Created'])

###  Multi-Rejection Counts
**Determine the amount of rejections for each project**

In [109]:
# creating a 'df' to count how many times a job was rejected (if at all)
improvement_counts_df = (improvements_df.groupby("Claim #").count())

# resets the 'claim #' from being the index
improvement_counts_df.reset_index(inplace=True)

# renaming the 'improvement_counts'to make it easier to merge
improvement_counts_df = improvement_counts_df.rename(columns = {"Created" : "Scope Rejections"})

### Latest Rejections
**Determine the most recent rejection for any project**

In [110]:
# 'unique_improv_dates_df' holds most recent 'created' dates for improvements;
# 'idmax()' provides the most current date
latest_rejection_df = improvements_df.loc[
    improvements_df.groupby('Claim #')['Created'].idxmax()]

# renaming the 'unique improvements' df to make it easier to merge
latest_rejection_df = latest_rejection_df.rename(columns={"Created": "Rejection Date"})

# Merge Data

## Merge 'All Projects' and 'Multi-Rejection' dataframes
**Merging dfs on the shared 'Claim #' Column**

In [111]:
first_merge_df = pd.merge(project_df, improvement_counts_df, how='left', on='Claim #')

# if the project as not rejected, fill the 'null' values with 0.
first_merge_df['Scope Rejections'] = first_merge_df['Scope Rejections'].fillna(0)

del first_merge_df['Job #_y']

## Combine with 'Latest Rejections'
**Merging dfs on the shared 'Claim #' Column**

In [112]:
second_merged_df = pd.merge(first_merge_df, latest_rejection_df, how='left', on='Claim #')

del second_merged_df['Job #']

## Combine with 'Production' Data

In [113]:
final_merged_df = pd.merge(second_merged_df, production_df, how='left', on='Claim #')
del final_merged_df['Job #_x'], final_merged_df['Branch_y']

# confirm 'OB completed date' is not on the production csv. 

final_merged_df.head()

,Claim #,Branch_x,Claim # Date,FTA Scope. Req Date,Submit for Estimate Date,[OB] Created Scope Calc,[B] Created Estimate Date,Job Submittal Date,[B] - Date Approved by BC,[OB] Completed_x,...,[OB] Completed_y,Permit Applied [A],Order Date,Permit Received,OA Date,Invoice Date,Ntfd H.O. Dlvry,Dlvry Start,Ntfd H.O. Start,Roof Start
0,60003552120,FCO,2019-06-09,2019-06-18,2019-06-19,2019-06-24,2019-06-25,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,165421V00,KCI,2018-08-19,2019-03-26,2019-03-26,2019-03-29,2019-03-29,2019-05-06,2019-05-08,2019-05-20,...,2019-05-20 16:38:00,2019-05-22,2019-05-30,2019-05-22,2019-05-30 10:20:00,2019-05-31,2019-05-30,2019-05-31,2019-05-31,2019-06-03
2,825102185,OMA,2018-10-12,2019-03-23,2019-04-04,2019-04-08,2019-04-08,2019-04-11,2019-04-12,2019-04-12,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,80-212-6908HO,OMA,2018-10-18,2019-03-23,2019-04-09,2019-04-11,2019-04-12,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,3011775484,FCO,2018-10-25,2019-04-19,2019-04-24,2019-04-25,2019-04-25,2019-05-30,2019-06-03,2019-06-03,...,2019-06-03 16:32:00,2019-06-20,2019-06-18,2019-06-21,2019-06-18 09:00:00,2019-06-19,2019-06-18,2019-06-19,2019-06-21,2019-06-24


## Cleanup Merged Data

In [120]:
all_project_df = final_merged_df[['Claim #', 'Job #', 'Branch_x', 'Scope Rejections', 'Claim Status', 'Claim # Date', 'FTA Scope. Req Date', 'Rejection Date', 'Submit for Estimate Date', '[B] Created Estimate Date', '[OB] Created Scope Calc',
                                  'Job Submittal Date', '[B] - Date Approved by BC', '[OB] Completed_x', 'Permit Applied [A]', 'Order Date', 'Permit Received', 'OA Date', 'Invoice Date', 'Ntfd H.O. Dlvry', 'Dlvry Start', 'Ntfd H.O. Start', 'Roof Start']]

# renaming the columnns to be able to read easier
all_project_df = all_project_df.rename(columns={
    'Branch_x': 'Branch',
    'FTA Scope. Req Date': 'Claim # Collected',
    'Rejection Date': 'FTA Scope Rejected',
    'Submit for Estimate Date': 'FTA Scope Completed',
    '[B] Created Estimate Date':'BC Estimate Completed',
    '[OB] Created Scope Calc': 'OB Scope Completed',
    'Job Submittal Date': 'Sup Submitted Job',
    '[B] - Date Approved by BC': 'BC Approved',
    '[OB] Completed_x': 'OB Order Built',
    'Permit Applied [A]': 'PA Permit Applied',
    'Order Date': 'GM Order Created',
    'Permit Received': 'PA Permit Processed',
    'OA Date': 'PA OA Processed',
    'Invoice Date':'PA Invoice Created',
    'Ntfd H.O. Dlvry': 'PA Notify of Delivery',
    'Dlvry Start': 'Delivery Start',
    'Ntfd H.O. Start': 'PA Notify of Start'
})

# Comparing Data

In [125]:
all_project_df.head()

,Claim #,Job #,Branch,Scope Rejections,Claim Status,Claim # Date,Claim # Collected,FTA Scope Rejected,FTA Scope Completed,BC Estimate Completed,...,OB Order Built,PA Permit Applied,GM Order Created,PA Permit Processed,PA OA Processed,PA Invoice Created,PA Notify of Delivery,Delivery Start,PA Notify of Start,Roof Start
0,60003552120,NaN,FCO,0.0,NaN,2019-06-09,2019-06-18,NaT,2019-06-19,2019-06-25,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,165421V00,1932820,KCI,0.0,NaN,2018-08-19,2019-03-26,NaT,2019-03-26,2019-03-29,...,2019-05-20,2019-05-22,2019-05-30,2019-05-22,2019-05-30 10:20:00,2019-05-31,2019-05-30,2019-05-31,2019-05-31,2019-06-03
2,825102185,NaN,OMA,1.0,NaN,2018-10-12,2019-03-23,2019-04-05,2019-04-04,2019-04-08,...,2019-04-12,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,80-212-6908HO,NaN,OMA,1.0,NaN,2018-10-18,2019-03-23,2019-04-10,2019-04-09,2019-04-12,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,3011775484,1934497,FCO,0.0,NaN,2018-10-25,2019-04-19,NaT,2019-04-24,2019-04-25,...,2019-06-03,2019-06-20,2019-06-18,2019-06-21,2019-06-18 09:00:00,2019-06-19,2019-06-18,2019-06-19,2019-06-21,2019-06-24


Index(['Claim #', 'Job #', 'Branch', 'Scope Rejections', 'Claim Status',
       'Claim # Date', 'Claim # Collected', 'FTA Scope Rejected',
       'FTA Scope Completed', 'BC Estimate Completed', 'OB Scope Completed',
       'Sup Submitted Job', 'BC Approved', 'OB Order Built',
       'PA Permit Applied', 'GM Order Created', 'PA Permit Processed',
       'PA OA Processed', 'PA Invoice Created', 'PA Notify of Delivery',
       'Delivery Start', 'PA Notify of Start', 'Roof Start'],
      dtype='object')

## Gathering Date Differences
**Finding any 'Submit for Estimate' and 'Rejection Date' Subsitutions**

In [173]:
# list to store the 'date diffs' value for each step'
claim_num = []
job_num = []
rep_claim_diff = []
fta_scope_diff = []
ob_scope_diff = []
bc_estimate_diff = []
sup_pfynr_diff = []
bc_approval_diff = []
ob_order_build_diff = []
gm_create_order_diff = []
pa_oa_processed_diff = []
pa_invoice_diff = []
total_days = []

# these are being recorded, but don't necessarily apply to the project workflow.
pa_permit_applied_diff = []
pa_permit_processed_diff = []
pa_notify_delivery_diff = []
pa_notify_start_diff = []


# iterating over the df to create 'date diff' variables
for index, row in all_project_df.iterrows():

    # creating 'date_diff' variables for each step in the workflow
    rep_claim_date_diff = float((row['Claim # Collected'] - row['Claim # Date']).days)

    # if the record has NOT had the FTA Scope Rejected...
    if row['Scope Rejections'] != 0:

        # if the bc estimate was created prior to July 16th...
        if row['BC Estimate Completed'] <= datetime(2019, 7, 15):

            # then compare the 'bc estimate' date to the 'ob scope calc' date
            # as well as 'ob scope' date to 'fta scope' date
            fta_date_diff = (row['FTA Scope Completed'] - row['Claim # Collected']).days
            ob_scope_date_diff = (row['OB Scope Completed'] - row['FTA Scope Completed']).days
            bc_estimate_date_diff = (row['BC Estimate Completed'] - row['OB Scope Completed']).days
            sup_pfynr_date_diff=(row['Sup Submitted Job'] - row['BC Estimate Completed']).days

        # if the record was addressed during the 'blip'...
        elif row['BC Estimate Completed'] == datetime(2019, 7, 16):
            
            # then compare the 'bc estimate' to the 'blip' date, and the 'ob scope' date to the new 'bc date'
            fta_date_diff = (row['FTA Scope Completed'] - row['Claim # Collected']).days
            bc_estimate_date_diff = (row['BC Estimate Completed'] - datetime.datetime(2019, 7, 15)).days
            ob_scope_date_diff = (row['OB Scope Completed'] - row['BC Estimate Completed']).days
            sup_pfynr_date_diff=(row['Sup Submitted Job'] - row['OB Scope Completed']).days

        # if the bc estimate was created after the 'blip' on July 16th...
        else:
            
            # then use the new workflow dates to compare the date diffs
            fta_date_diff = (row['FTA Scope Completed'] - row['Claim # Collected']).days
            bc_estimate_date_diff = (row['BC Estimate Completed'] - row['FTA Scope Completed']).days
            ob_scope_date_diff = (row['OB Scope Completed'] - row['BC Estimate Completed']).days
            sup_pfynr_date_diff=(row['Sup Submitted Job'] - row['OB Scope Completed']).days
            
    # if the record HAS has the FTA Scope Rejected...
    else:
        
        # and was rejected prior to the 'blip'...
        if row['BC Estimate Completed'] <= datetime(2019, 7, 15):
            
            # use 'rejected' date and clarify sup date diffs
            fta_date_diff = (row['FTA Scope Rejected'] - row['Claim # Collected']).days
            bc_estimate_date_diff = (row['BC Estimate Completed'] - row['FTA Scope Completed']).days
            ob_scope_date_diff = (row['OB Scope Completed'] - row['FTA Scope Rejected']).days
            sup_pfynr_date_diff=(row['Sup Submitted Job'] - row['BC Estimate Completed']).days
        
        # if rejected after the 'blip'...
        else:
            
            # use 'rejected' date and clarify sup date diffs
            fta_date_diff = (row['FTA Scope Rejected'] - row['Claim # Collected']).days
            bc_estimate_date_diff = (row['BC Estimate Completed'] - row['FTA Scope Completed']).days
            ob_scope_date_diff = (row['OB Scope Completed'] - row['FTA Scope Rejected']).days
            sup_pfynr_date_diff = (row['Sup Submitted Job'] - row['OB Scope Completed']).days
    
    bc_approval_date_diff = (row['BC Approved'] - row['Sup Submitted Job']).days
    ob_orderbuild_date_diff = (row['OB Order Built'] - row['BC Approved']).days
    gm_create_order_date_diff = (row['GM Order Created'] - row['OB Order Built']).days
    pa_oa_processed_date_diff = (row['PA OA Processed'] - row['GM Order Created']).days
    pa_invoice_date_diff = (row['PA Invoice Created'] - row['PA OA Processed']).days
    
    # these provide the lead times of tasks not directly impacting the workflow.
    pa_permit_applied_date_diff = (row['PA Permit Applied'] - row['BC Approved']).days
    pa_permit_processed_date_diff = (row['PA Permit Processed'] - row['PA Permit Applied']).days
    pa_notify_delivery_date_diff = (row['Delivery Start'] - row['PA Notify of Delivery']).days
    pa_notify_start_date_diff = (row['Roof Start'] - row['PA Notify of Start']).days



    # adding up all of the 'date_diff' variables above and assigning to 'total_days_sum'
    day_diffs=[rep_claim_date_diff, fta_date_diff, ob_scope_date_diff, bc_estimate_date_diff,
               sup_pfynr_date_diff, bc_approval_date_diff, ob_orderbuild_date_diff, 
               gm_create_order_date_diff, pa_oa_processed_date_diff, pa_invoice_date_diff]
    
    total_days_sum = sum(day_diffs)
        
    # appending 'date diff' values to lists to create each df column
    claim_num.append(row["Claim #"])
    job_num.append(row["Job #"])
    rep_claim_diff.append(rep_claim_date_diff)
    fta_scope_diff.append(fta_date_diff)
    ob_scope_diff.append(ob_scope_date_diff)
    bc_estimate_diff.append(bc_estimate_date_diff)
    sup_pfynr_diff.append(sup_pfynr_date_diff)
    bc_approval_diff.append(bc_approval_date_diff)
    ob_order_build_diff.append(ob_orderbuild_date_diff)
    gm_create_order_diff.append(gm_create_order_date_diff)
    pa_oa_processed_diff.append(pa_oa_processed_date_diff)
    pa_invoice_diff.append(pa_invoice_date_diff)
    total_days.append(total_days_sum)
    pa_permit_applied_diff.append(pa_permit_applied_date_diff)
    pa_permit_processed_diff.append(pa_permit_processed_date_diff)
    pa_notify_delivery_diff.append(pa_notify_delivery_date_diff)
    pa_notify_start_diff.append(pa_notify_start_date_diff)

## Creating 'Workflow Days' df
**The days between each Teammate step in the workflow**

In [179]:
# creating the 'days_df' to hold all date values for each role in the project
days_df = pd.DataFrame({
    "claim_#": claim_num,
    "job_#": job_num,
    "rep_claim": rep_claim_diff,
    "fta_scope": fta_scope_diff,
    "bc_estimate": bc_estimate_diff,
    "ob_scope": ob_scope_diff,
    "sup_pfynr": sup_pfynr_diff,
    "bc_approval": bc_approval_diff,
    "ob_orderbuild": ob_order_build_diff,
    "gm_create_order": gm_create_order_diff,
    "pa_oa_process": pa_oa_processed_diff,
    "pa_invoice": pa_invoice_diff,
    "total_days": total_days
})

In [181]:
days_df.head(25)

,claim_#,job_#,rep_claim,fta_scope,bc_estimate,ob_scope,sup_pfynr,bc_approval,ob_orderbuild,gm_create_order,pa_oa_process,pa_invoice,total_days
0,60003552120,NaN,9.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,165421V00,1932820,219.0,NaN,3.0,NaN,38.0,2.0,12.0,10.0,0.0,0.0,NaN
2,825102185,NaN,162.0,12.0,0.0,4.0,3.0,1.0,0.0,NaN,NaN,NaN,NaN
3,80-212-6908HO,NaN,156.0,17.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3011775484,1934497,176.0,NaN,1.0,NaN,35.0,4.0,0.0,15.0,0.0,0.0,NaN
5,3011771813,1934612,231.0,5.0,0.0,6.0,3.0,1.0,3.0,9.0,NaN,NaN,NaN
6,926104,1934874,4.0,NaN,5.0,NaN,17.0,1.0,0.0,4.0,0.0,5.0,NaN
7,926109,1934876,4.0,9.0,4.0,1.0,17.0,1.0,0.0,4.0,0.0,2.0,42.0
8,926112,1934877,4.0,NaN,8.0,NaN,14.0,4.0,1.0,0.0,0.0,2.0,NaN
9,333036-GI,1934980,33.0,15.0,1.0,11.0,65.0,4.0,0.0,14.0,0.0,2.0,145.0


# Export Data

In [176]:
# 'projects' and 'workflow days' CSVs

final_merged_df.to_csv("data/project_table.csv", index=False)

days_df.to_csv("data/workflow_days.csv", index=False)
